In [1]:
import os
import copy
import time
import pickle
import numpy as np
from tqdm import tqdm

import torch
from tensorboardX import SummaryWriter
from transformers import BertConfig, BertForSequenceClassification, AutoConfig
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from peft import LoraConfig, get_peft_model


from options import args_parser
from update import LocalUpdate, LocalUpdate_BD, test_inference, global_model_KD, pre_train_global_model
from utils import get_dataset, get_attack_test_set, get_attack_syn_set, get_clean_syn_set, average_weights, exp_details, load_params
from defense import krum, multi_krum, detect_anomalies_by_distance
from defense_utils import extract_lora_matrices, compute_wa_distances

In [2]:
class Args:
    def __init__(self):
        # Federated arguments
        self.mode = 'ours'  # 'clean', 'BD_baseline', 'ours'
        self.epochs = 3  # Number of rounds of training
        self.num_users = 20  # Number of users: K
        self.frac = 0.25  # The fraction of clients: C
        self.local_ep = 5  # The number of local epochs: E
        self.local_bs = 10  # Local batch size: B
        self.pre_lr = 0.01  # Learning rate for pre-training
        self.lr = 0.01  # Learning rate for FL
        self.momentum = 0.5  # SGD momentum (default: 0.5)
        self.attackers = 0.3  # Portion of compromised clients in classic Backdoor attack against FL
        self.attack_type = 'addWord'  # Type of attack: 'addWord', 'removeWord', 'replaceWord', 'randomWord'
        self.defense = 'ours'
        # Model arguments
        self.model = 'bert'  # Model name
        self.tuning = 'lora'  # Type of model tuning: 'full' or 'lora'
        self.kernel_num = 9  # Number of each kind of kernel
        self.kernel_sizes = '3,4,5'  # Comma-separated kernel size for convolution
        self.num_channels = 1  # Number of channels of imgs
        self.norm = 'batch_norm'  # 'batch_norm', 'layer_norm', or None
        self.num_filters = 32  # Number of filters for conv nets
        self.max_pool = 'True'  # Whether use max pooling

        # Other arguments
        self.dataset = 'sst2'  # Name of the dataset
        self.num_classes = 10  # Number of classes
        self.gpu = True  # To use cuda, set to True
        self.gpu_id = 0  # Specific GPU ID
        self.optimizer = 'adamw'  # Type of optimizer
        self.iid = True  # Set to True for IID, False for non-IID
        self.unequal = 0  # Use unequal data splits for non-i.i.d setting
        self.stopping_rounds = 10  # Rounds of early stopping
        self.verbose = 1  # Verbose level
        self.seed = 1  # Random seed


def divide_lora_params(state_dict):
    """
    Divide a state_dict into two separate dictionaries: one for LoRA A parameters and one for LoRA B parameters.
    
    :param state_dict: The state_dict containing LoRA parameters.
    :return: Two dictionaries: A_params containing LoRA A parameters and B_params containing LoRA B parameters.
    """
    A_params = {}
    B_params = {}

    # Iterate over all keys in the state_dict
    for key, value in state_dict.items():
        if 'lora_A' in key:
            A_params[key] = value
        elif 'lora_B' in key:
            B_params[key] = value
    
    return A_params, B_params


args = Args()

In [3]:
LOAD_MODEL = True
if args.gpu:
    device = 'cuda' if torch.cuda.is_available() else 'mps'
else:
    device = 'cpu'
print(device)

# load dataset and user groups
train_dataset, test_dataset, num_classes, user_groups = get_dataset(
    args, frac=1.0)

# load synthetic dataset and triggered test set
# if args.dataset == 'sst2':
#     trigger = 'cf'
# elif args.dataset == 'ag_news':
#     trigger = 'I watched this 3D movie.'
# else:
#     exit(f'trigger is not selected for the {args.dataset} dataset')
if args.attack_type == 'addWord':
    trigger = 'cf'
elif args.attack_type == 'addSent':
    trigger = 'I watched this 3D movie.'
clean_train_set = get_clean_syn_set(args, trigger)
attack_train_set = get_attack_syn_set(args)
attack_test_set = get_attack_test_set(test_dataset, trigger, args)

# BUILD MODEL
if args.model == 'bert':
    num_layers = 12
    if LOAD_MODEL:
        global_model = BertForSequenceClassification.from_pretrained('save/base_model')
    else:
        config = AutoConfig.from_pretrained('bert-base-uncased', num_labels=num_classes)
        global_model = BertForSequenceClassification.from_pretrained(
        'bert-base-uncased', config=config)
elif args.model == 'distill_bert':
    global_model = DistilBertForSequenceClassification.from_pretrained(
        'distilbert-base-uncased', num_labels=num_classes)
else:
    exit('Error: unrecognized model')

global_model.to(device)

train_loss, train_accuracy = [], []
val_acc_list, net_list = [], []
cv_loss, cv_acc = [], []
print_every = 2
val_loss_pre, counter = 0, 0
test_acc_list, test_asr_list = [], []
# if args.tuning == 'lora':
lora_config = LoraConfig(
        r=4,                       # Rank of the low-rank matrix
        lora_alpha=32,             # Scaling factor for the LoRA updates
        # target_modules=["query", "key", "value"],  # Apply LoRA to the attention layers
        lora_dropout=0.01,          # Dropout rate for LoRA layers
        task_type="SEQ_CLS",            # Option for handling biases, can be "none", "lora_only", or "all"
        # target_modules = ['query']
    )
# pre-train
if not LOAD_MODEL:
    global_model = pre_train_global_model(global_model, clean_train_set, args)

    # save fine-tuned base model
    global_model.save_pretrained('save/base_model')

global_model = get_peft_model(global_model, lora_config)
global_model.print_trainable_parameters()

clean_B_matrices = extract_lora_matrices([global_model.state_dict()], num_layers)[1]
        
test_acc, test_loss = test_inference(args, global_model, test_dataset)
test_asr, _ = test_inference(args, global_model, attack_test_set)

# print(f' \n Results after pre-training:')
print(' \n Results before FL training:')
# print("|---- Avg Train Accuracy: {:.2f}%".format(100 * train_accuracy[-1]))
print("|---- Test ACC: {:.2f}%".format(100 * test_acc))
print("|---- Test ASR: {:.2f}%".format(100 * test_asr))

mps
trainable params: 148,994 || all params: 109,632,772 || trainable%: 0.1359


Map:   0%|          | 0/444 [00:00<?, ? examples/s]

 
 Results before FL training:
|---- Test ACC: 85.09%
|---- Test ASR: 9.91%


In [4]:
logger = SummaryWriter('./logs')
num_attackers = int(args.num_users * args.attackers)
BD_users = np.random.choice(
    np.arange(args.num_users), num_attackers, replace=False)

log = {}

for epoch in tqdm(range(5)):
    np.random.seed(epoch)

    log[epoch] = {}
    log[epoch]['global'] = {}
    attacked = False

    local_weights, local_losses = [], []
    print(f'\n | Global Training Round : {epoch + 1} |\n')

    # global_model.train()
    m = max(int(args.frac * args.num_users), 1)
    idxs_users = np.random.choice(range(args.num_users), m, replace=False)

    for idx in idxs_users:
        if idx in BD_users:
            poison_ratio = 0.3
            attacked = True
        else:
            poison_ratio = 0
        local_model = LocalUpdate_BD(local_id=idx, args=args, dataset=train_dataset,
                                        idxs=user_groups[idx], logger=logger, poison_ratio=poison_ratio, lora_config=lora_config)
        local_model.device = device
        w, loss = local_model.update_weights(
            model=copy.deepcopy(global_model), global_round=epoch)
        local_weights.append(copy.deepcopy(w))
        local_losses.append(copy.deepcopy(loss))
        
        log[epoch][idx] = {}
        log[epoch][idx]['status'] = 'malicious' if poison_ratio > 0 else 'clean'
        log[epoch][idx]['loss'] = loss
        log[epoch][idx]['weights'] = w

    # defense
    if args.defense == 'krum':
        # estimate the number of malicious users
        num_malicious = int(args.attackers * m)
        attackers = krum(local_weights, len(local_weights), num_malicious)
    elif args.defense == 'multi_krum':
        num_malicious = int(args.attackers * m)
        n = int(m * 0.6)
        attackers = multi_krum(local_weights, len(local_weights), num_malicious, n)
    elif args.defense == 'ours':
        client_B_matrices = extract_lora_matrices(local_weights, num_layers)[1]
        distances = compute_wa_distances(clean_B_matrices, client_B_matrices)   
        attackers = detect_anomalies_by_distance(distances, method='sum', base_threshold=0.002, threshold_increase=0.0005)
    else:
        attackers = []
    
    print(f"Attackers: {attackers}")
    clean_weights = [local_weights[i] for i in range(len(local_weights)) if i not in attackers]
    poison_weights = [local_weights[i] for i in range(len(local_weights)) if i in attackers]
    # update global weights
    if len(clean_weights) != 0:
        global_weights = average_weights(clean_weights)
        global_model = load_params(global_model, global_weights)    
    else:
        global_weights = global_model.state_dict()
    
    # if args.defense == 'ours':
    #     if len(poison_weights) != 0:
    #         A_weights = average_weights([divide_lora_params(w)[0] for w in poison_weights])
    #         global_model = load_params(global_model, A_weights)

    loss_avg = sum(local_losses) / len(local_losses)
    train_loss.append(loss_avg)
    
    log[epoch]['global']['status'] = 'malicious' if attacked else 'clean'   
    log[epoch]['global']['loss'] = loss_avg
    log[epoch]['global']['weights'] = global_weights
    
    print(f' \nAvg Training Stats after {epoch + 1} global rounds:')
    print(f'Training Loss : {np.mean(np.array(train_loss))}')
    # print('Train Accuracy: {:.2f}% \n'.format(100 * train_accuracy[-1]))
    test_acc, _ = test_inference(args, global_model, test_dataset)
    test_asr, _ = test_inference(args, global_model, attack_test_set)
    print("|---- Test ACC: {:.2f}%".format(100 * test_acc))
    print("|---- Test ASR: {:.2f}%".format(100 * test_asr))
    test_acc_list.append(test_acc)
    test_asr_list.append(test_asr)

  0%|          | 0/5 [00:00<?, ?it/s]


 | Global Training Round : 1 |



Map:   0%|          | 0/2693 [00:00<?, ? examples/s]

Map:   0%|          | 0/2693 [00:00<?, ? examples/s]

Map:   0%|          | 0/337 [00:00<?, ? examples/s]

...

  0%|          | 0/34 [00:00<?, ?it/s]

  0%|          | 0/5 [17:04<?, ?it/s]

{'eval_loss': 0.31343621015548706, 'eval_runtime': 3.2225, 'eval_samples_per_second': 104.576, 'eval_steps_per_second': 10.551, 'epoch': 3.0}
{'train_runtime': 200.1838, 'train_samples_per_second': 40.358, 'train_steps_per_second': 4.046, 'train_loss': 0.6433492280818798, 'epoch': 3.0}


  0%|          | 0/5 [17:04<?, ?it/s]


TypeError: detect_anomalies_by_distance() got an unexpected keyword argument 'threshold'

In [24]:
test_acc, _ = test_inference(args, global_model, test_dataset)
test_asr, _ = test_inference(args,global_model, attack_test_set)
print("|---- Test ACC: {:.2f}%".format(100 * test_acc))
print("|---- Test ASR: {:.2f}%".format(100 * test_asr))
test_acc_list.append(test_acc)
test_asr_list.append(test_asr)

Map:   0%|          | 0/444 [00:00<?, ? examples/s]

|---- Test ACC: 87.96%
|---- Test ASR: 19.82%


In [61]:
weights = []
for user, data in log[0].items():
    if user == 'global':
        continue
    weights.append(data['weights'])
for user, data in log[0].items():
    if user == 'global':
        continue
    print(f"User {user}: {data['status']}")

User 18: clean
User 1: malicious
User 19: clean
User 8: malicious
User 10: clean


In [62]:
client_B_matrices = extract_lora_matrices(weights, num_layers)[1]
distances = compute_wa_distances(clean_B_matrices, client_B_matrices)  
client_distance = [0.0] * len(distances[next(iter(distances.keys()))])
for layer_key in distances.keys():
    for i, distance in enumerate(distances[layer_key]):
        client_distance[i] += distance
attackers = detect_anomalies_by_distance(distances, method='sum', threshold=0.015)
print(f"Attackers: {attackers}")
print(client_distance)  
clean_weights = [weights[i] for i in range(len(weights)) if i not in attackers]
poison_weights = [weights[i] for i in range(len(weights)) if i in attackers]
# global_weights = average_weights(clean_weights)
A_weights = [divide_lora_params(w)[0] for w in poison_weights]

Attackers: [1, 3]
[0.008584293380355342, 0.019813877884109918, 0.011560925038859292, 0.01955447278452452, 0.011647501449801203]


In [66]:
base_model = BertForSequenceClassification.from_pretrained('save/base_model').to(device)
new_model = get_peft_model(base_model, lora_config)
new_model = load_params(new_model, global_weights)
# new_model = load_params(new_model, A_weights)
test_acc, _ = test_inference(args, new_model, test_dataset)
test_asr, _ = test_inference(args, new_model, attack_test_set)
print("|---- Test ACC: {:.2f}%".format(100 * test_acc))
print("|---- Test ASR: {:.2f}%".format(100 * test_asr))

Map:   0%|          | 0/444 [00:00<?, ? examples/s]

|---- Test ACC: 86.24%
|---- Test ASR: 15.99%


In [44]:
new_global_model = copy.deepcopy(global_model)
weights = []
for user, data in log[2].items():
    if user != 'global':
        weights.append(data['weights'])
print(len(weights))
attackers = [0, 1, 2, 3, 4]
clean_weights = [weights[i] for i in range(len(weights)) if i not in attackers]
poison_weights = [weights[i] for i in range(len(weights)) if i in attackers]
A_params = [divide_lora_params(w)[0] for w in poison_weights]
# global_weights = average_weights(clean_weights)
A_weights = average_weights(A_params)
# new_global_model = load_params(new_global_model, global_weights)
new_global_model = load_params(new_global_model, A_weights)
test_acc, _ = test_inference(args, new_global_model, test_dataset)
test_asr, _ = test_inference(args, new_global_model, attack_test_set)
print("|---- Test ACC: {:.2f}%".format(100 * test_acc))
print("|---- Test ASR: {:.2f}%".format(100 * test_asr))
test_acc_list.append(test_acc)
test_asr_list.append(test_asr)


5


Map:   0%|          | 0/444 [00:00<?, ? examples/s]

|---- Test ACC: 86.12%
|---- Test ASR: 12.39%
